## Resumen de limpieza

Hemos revisado de cada uno de los archivos los nulos y los duplicados.

- Se ha encontrado un duplicado en el archivo de 'actor.csv', para ello se filtró por nombre y apellido ya que el id era diferente. En esa misma tabla unimos el nombre y apellido en una única columna para facilitar la identificación de actores y actrices en las prox búsquedas.

- Nulos solamente había en 'film.csv' en toda una columna y la hemos eliminado. En esta misma tabla hemos realizado varios cambios. En primer lugar hemos visto que el año de salida era en todas el 2006, por lo que es un dato mal introducido y eliminamos esa columna. Como en el videoclub todas las películas son en inglés hemos decidido eliminar las columnas 'original_language_id' y 'language_id'.

- En todas las tablas que la contenían, hemos eliminado la columna 'last_update' ya que constaba siempre la misma fecha y no lo consideramos info relevante.

- Para la tabla de 'inventory' eliminamos también la columna de las tiendas porque no será útil para el nuevo negocio. Nos damos cuenta durante la limpieza de que la info ofrecida solo constan las 233 primeras películas.

- Limpiamos la tabla 'old_hdd' de tal manera que sólo queden el nombre completo de los actores y en nombre de la película correspondiente. Esto nos es útil para crear una nueva columna en 'films' con los actores correspondientes a cada película.

- Revisando 'rental' eliminamos la columna de 'staff_id'. Como podemos observar, en la columna de inventory_id hemos descubierto que hay valores más altos que los que tenemos de registro en la tabla inventory. Por lo que suponemos que no se nos ha ofrecido la información completa de esa tabla. Asimismo, vemos que hay 1000 registros pero en el inv_id los valores llegan a 1001 por lo que, revisando las filas, encontramos que hay un registro que falta (entre el 320 y el 322). El uso de esta tabla tal cual nos daría error en sql para poder hacer las relaciones por lo que tenemos dos opciones: o bien eliminamos todas las filas en las que el valor de la columna inventory_id sea mayor que 1001 (se puede realizar tanto con py o sql), o podemos por otro lado guardar los datos antiguos limpios por un posible uso futuro de comparativas y resetear la tabla rental para tener nuevos valores de los registros una vez abra.


In [61]:
import pandas as pd
import numpy as np
from funciones import *

import warnings
warnings.filterwarnings('ignore')

In [62]:
act = pd.read_csv('../../not/actor.csv')
cat = pd.read_csv('../../not/category.csv')
fil = pd.read_csv('../../not/film.csv')
inv = pd.read_csv('../../not/inventory.csv')
lan = pd.read_csv('../../not/language.csv')
old = pd.read_csv('../../not/old_HDD.csv')
ren = pd.read_csv('../../not/rental.csv')

act.head()


,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [63]:
act.shape

(200, 4)

In [64]:
nulitos(act)

Series([], dtype: int64)


In [65]:
act.duplicated().unique()

array([False])

In [66]:
act = act.drop('last_update', axis=1)

In [67]:
act.loc[act[['first_name', 'last_name']].duplicated(keep=False)]

,actor_id,first_name,last_name
100,101,SUSAN,DAVIS
109,110,SUSAN,DAVIS


In [68]:
act.drop_duplicates(['first_name', 'last_name'], keep=False, inplace=True)

In [69]:
act['full_name'] = act['first_name'].str.cat(act['last_name'], sep = ' ')
act.drop(['first_name', 'last_name'], axis=1, inplace=True)
act

,actor_id,full_name
0,1,PENELOPE GUINESS
1,2,NICK WAHLBERG
2,3,ED CHASE
3,4,JENNIFER DAVIS
4,5,JOHNNY LOLLOBRIGIDA
...,...,...
195,196,BELA WALKEN
196,197,REESE WEST
197,198,MARY KEITEL
198,199,JULIA FAWCETT


In [70]:
act['actor_id'].duplicated().unique()

array([False])

In [71]:
cat.head()


,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27


In [72]:
nulitos(cat)

Series([], dtype: int64)


In [73]:
cat.shape

(16, 3)

In [74]:
cat = cat.drop('last_update', axis=1)
cat['category_id'].duplicated().unique()

array([False])

In [75]:
fil.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [76]:
fil['release_year'].value_counts()

release_year
2006    1000
Name: count, dtype: int64

In [77]:
fil['film_id'].max()

1000

In [78]:
fil.shape

(1000, 13)

In [79]:
nulitos(fil)

original_language_id    1000
dtype: int64


In [80]:
fil['language_id'].value_counts()

language_id
1    1000
Name: count, dtype: int64

In [81]:
fil = fil.drop(['last_update', 'original_language_id', 'language_id', 'release_year'], axis=1)


In [82]:
fil['film_id'].duplicated().unique()
fil['title'].duplicated().unique()

array([False])

In [83]:
fil['rating'].unique()

array(['PG', 'G', 'NC-17', 'PG-13', 'R'], dtype=object)

In [84]:
fil['length'].max()

185

In [85]:
fil.head()

,film_id,title,description,rental_duration,rental_rate,length,replacement_cost,rating,special_features
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes"
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,3,4.99,48,12.99,G,"Trailers,Deleted Scenes"
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes"
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes"
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,6,2.99,130,22.99,G,Deleted Scenes


In [86]:
inv.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [87]:
inv = inv.drop(['last_update','store_id'], axis=1)
inv['inventory_id'].duplicated().unique()

array([False])

In [88]:
inv['film_id'].max() #el inv sólo tiene stock de las primeras 223 pelis

223

In [89]:
inv['inventory_id'].max()

1000

In [90]:
inv.head()

,inventory_id,film_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [91]:
lan.head()

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19


In [92]:
nulitos(lan)

Series([], dtype: int64)


In [93]:
lan = lan.drop('last_update', axis=1)
lan['language_id'].duplicated().unique()


array([False])

In [94]:
lan['name'].value_counts()

name
English     1
Italian     1
Japanese    1
Mandarin    1
French      1
German      1
Name: count, dtype: int64

In [95]:
old.head()

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14


In [96]:
old.tail()

,first_name,last_name,title,release_year,category_id
995,GOLDIE,BRODY,COMANCHEROS ENEMY,2006,3
996,GOLDIE,BRODY,DAISY MENAGERIE,2006,14
997,GOLDIE,BRODY,DESERT POSEIDON,2006,11
998,GOLDIE,BRODY,EVERYONE CRAFT,2006,9
999,GOLDIE,BRODY,EXORCIST STING,2006,15


In [97]:
old.shape

(1000, 5)

In [98]:
nulitos(old)

Series([], dtype: int64)


In [99]:
old['category_id'].duplicated().unique()

array([False,  True])

In [100]:
old['full_name'] = old['first_name'].str.cat(old['last_name'], sep = ' ')
old.drop(['first_name', 'last_name', 'category_id', 'release_year'], axis=1, inplace=True)

old

,title,full_name
0,ACADEMY DINOSAUR,PENELOPE GUINESS
1,ANACONDA CONFESSIONS,PENELOPE GUINESS
2,ANGELS LIFE,PENELOPE GUINESS
3,BULWORTH COMMANDMENTS,PENELOPE GUINESS
4,CHEAPER CLYDE,PENELOPE GUINESS
...,...,...
995,COMANCHEROS ENEMY,GOLDIE BRODY
996,DAISY MENAGERIE,GOLDIE BRODY
997,DESERT POSEIDON,GOLDIE BRODY
998,EVERYONE CRAFT,GOLDIE BRODY


In [101]:
# old['film_id'] = fil['film_id'].copy()
# old = old.merge(fil, on=(['film_id']), how='left')

In [102]:
#old.drop(['first_name', 'last_name', 'category_id', 'release_year'], axis=1, inplace=True)

old

,title,full_name
0,ACADEMY DINOSAUR,PENELOPE GUINESS
1,ANACONDA CONFESSIONS,PENELOPE GUINESS
2,ANGELS LIFE,PENELOPE GUINESS
3,BULWORTH COMMANDMENTS,PENELOPE GUINESS
4,CHEAPER CLYDE,PENELOPE GUINESS
...,...,...
995,COMANCHEROS ENEMY,GOLDIE BRODY
996,DAISY MENAGERIE,GOLDIE BRODY
997,DESERT POSEIDON,GOLDIE BRODY
998,EVERYONE CRAFT,GOLDIE BRODY


In [103]:
ren.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [104]:
ren.shape

(1000, 7)

In [105]:
nulitos(ren)

Series([], dtype: int64)


In [106]:
ren.drop(['last_update', 'staff_id'], axis=1, inplace=True)

In [107]:
ren.duplicated().unique()

array([False])

In [108]:
ren['rental_id'].duplicated().unique()
ren.head()

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21


In [109]:
fil['actors'] = '' 
def actors(peli): 
    act = []
    for i in old.index:
        if old.title[i] == peli:
            name = old.full_name[i]
            act.append(name)
    return ', '.join(act)

In [110]:
for i in fil.index:
    fil['actors'][i] = actors(fil.title[i])


# función reciclada adaptada a nuestro código (gracias).

In [111]:
nulitos(fil)

Series([], dtype: int64)


In [112]:
fil.actors.value_counts()

actors
                                            386
SANDRA KILMER                                13
VIVIEN BERGEN                                13
MILLA PECK                                   13
TOM MCKELLEN                                 12
                                           ... 
CAMERON STREEP, ALEC WAYNE, TOM MCKELLEN      1
CUBA OLIVIER, ALEC WAYNE                      1
FRED COSTNER, RIP CRAWFORD                    1
ZERO CAGE, LUCILLE TRACY                      1
SANDRA KILMER, VAL BOLGER                     1
Name: count, Length: 284, dtype: int64

In [113]:
fil.loc[fil['actors'] == '', 'actors'] = 'VARIOUS'  #relleno de vacíos

In [114]:
fil

,film_id,title,description,rental_duration,rental_rate,length,replacement_cost,rating,special_features,actors
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes","PENELOPE GUINESS, CHRISTIAN GABLE, LUCILLE TRA..."
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",BOB FAWCETT
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes","NICK WAHLBERG, BOB FAWCETT, CAMERON STREEP"
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",VARIOUS
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,6,2.99,130,22.99,G,Deleted Scenes,VARIOUS
...,...,...,...,...,...,...,...,...,...,...
995,996,YOUNG LANGUAGE,A Unbelieveable Yarn of a Boat And a Database ...,6,0.99,183,9.99,G,"Trailers,Behind the Scenes","ED CHASE, JULIA MCQUEEN"
996,997,YOUTH KICK,A Touching Drama of a Teacher And a Cat who mu...,4,0.99,179,14.99,NC-17,"Trailers,Behind the Scenes","SANDRA KILMER, VAL BOLGER"
997,998,ZHIVAGO CORE,A Fateful Yarn of a Composer And a Man who mus...,6,0.99,105,10.99,NC-17,Deleted Scenes,UMA WOOD
998,999,ZOOLANDER FICTION,A Fateful Reflection of a Waitress And a Boat ...,5,2.99,101,28.99,R,"Trailers,Deleted Scenes",VARIOUS


In [115]:
ren

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21
...,...,...,...,...,...
995,997,2005-05-31 00:08:25,4243,216,2005-06-02 00:17:25
996,998,2005-05-31 00:16:57,3395,389,2005-06-01 22:41:57
997,999,2005-05-31 00:25:10,4433,413,2005-06-03 06:05:10
998,1000,2005-05-31 00:25:56,1774,332,2005-06-08 19:42:56


Como podemos observar, en la columna de inventory_id hemos descubierto que hay valores más altos que los que tenemos de registro en la tabla inventory. Por lo que suponemos que no se nos ha ofrecido la información completa de esa tabla. Asimismo, vemos que hay 1000 registros pero en el inv_id los valores llegan a 1001 por lo que, revisando las filas, encontramos que hay un registro que falta (entre el 320 y el 322). El uso de esta tabla tal cual nos daría error en sql para poder hacer las relaciones por lo que tenemos dos opciones: o bien eliminamos todas las filas en las que el valor de la columna inventory_id sea mayor que 1001 (se puede realizar tanto con py o sql), o podemos por otro lado guardar los datos antiguos limpios por un posible uso futuro de comparativas y resetear la tabla rental para tener nuevos valores de los registros una vez abra.

In [116]:
# fil['categories'] = '' #creamos columna vacía
# def categories(peli): #creamos función que saque los actores de esa peli
#     act = []
#     for i in old.index:
#         if old.title[i] == peli:
#             name = old.full_name[i]
#             act.append(name)
#     return ', '.join(act)

Así se haría con py la eliminación de las filas de rental sin toda la info.

In [117]:
#ren = ren.drop(ren[ren['inventory_id'] >1000].index)

In [118]:
act.to_csv('../../not/actor_clean.csv', index=False)
fil.to_csv('../../not/film_clean.csv', index=False)
inv.to_csv('../../not/inventory_clean.csv', index=False)
old.to_csv('../../not/hdd_clean.csv', index=False)
ren.to_csv('../../not/rental_clean.csv', index=False)
